# ganram
# GAN-BASED DATA AUGMENTATION for RAMAN SPECTRA

#### (KRG group: https://myweb.uoi.gr/nkourkou/)

#### * in this notebook (opt) denotes a parameter that can be optimised

# Directory structure

```
project ganram/
│   readme.md
│   project.ipynb
│   scavenging_PCA.ipynb
│
└───data/
│       input.csv
│   
└───output/
│       └───data/
│               csv/
│                  output_synthetic.scv
│               evolution/
│                        image_at_epoch_000X_0000.png (images)
│               cp/
│                 checkpoints_X.scv
│               generated_samples/
│                                generated_sample_0000 (images)
│   
└───models/
│         generator_model.ckpt
│       
└───training_checkpoints/
│                       ckpt-X.data-00000-of-0000X (checkpoints)
        
```

# Library Imports

In [ ]:
import os
import glob
import scipy as sp
from scipy import signal
import pandas as pd
import numpy as np
import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras as keras
from keras import layers
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU, GaussianNoise
import time
import matplotlib.pyplot as plt
%matplotlib inline

import itertools
from frechetdist import frdist
import random


# Data Loading and Preprocessing

In [ ]:
REMOVE_FIRST_COLUMN = True                  # Remove first column from the dataframe if it has IDs, names, etc.) 
df = pd.read_csv('data/tibia_bones_raw_1800.csv')

if REMOVE_FIRST_COLUMN:
    df = df.drop("Raman_shift", axis=1)              # Drop the "Raman_shift" column from the dataframe

print("Successfuly loaded the dataset")
df                                          # Showing the data

In [ ]:
# Keeping the index labels (wavelength) for later use (for plotting etc.)

df.columns= df.columns.astype(float)
column_labels = df.columns.tolist
column_labels

In [ ]:
# Converting the data to numpy array
data_raw = df.to_numpy()                    
print(f'Raw data shape: {data_raw.shape}')  # Sanity check of shapes

## Data Preprocessing (low pass filter) and train/test split

In [ ]:
# If the signal is very complex or noisy (like EEG etc.) we can use LPF
 

# data_raw = df.to_numpy()        
# # Getting the filter coefficients of the low-pass butterworth filter 
# b, a = signal.butter(2, 0.3, 'low', analog = False)                 # Order of filter and cutoff frequency
# data_denoised = signal.filtfilt(b, a, data_raw)                     # Applying the filter on the data, axis=-1 (row-wise) by default 
# #data_denoised = data_raw

# # Data distribution modification
# means = np.average(data_denoised, axis=0).reshape(1, -1)            # Calculating data mean (column-wise); mean of each feature
# std_dev  = np.std(data_denoised, axis= 0).reshape(1, -1)            # Calculating data standard deviation
# data_processed = (data_denoised - means) / std_dev                  # Data normalization (x-u)/sigma
# print(data_processed)

# # Slicing the data
# train_data = data_processed[:int(data_processed.shape[0]*0.8), :]                # Doing the training split
# test_data = data_processed[int(data_processed.shape[0]*0.8):, :]                 # Doing the test split
# print("Train dataset shape: {}".format(train_data.shape))
# print("Test dataset shape: {}".format(test_data.shape))

## Data Preprocessing (train/test split)

In [ ]:
data_processed = data_raw 

# Slicing the data

train_data = data_processed[:int(data_processed.shape[0]*0.8), :]                # Doing the training split
test_data = data_processed[int(data_processed.shape[0]*0.8):, :]                 # Doing the test split
print("Train dataset shape: {}".format(train_data.shape))
print("Test dataset shape: {}".format(test_data.shape))

## Visualizing Data Samples

In [ ]:
plt.plot(df.T.index.astype(float), train_data[10])
plt.show()


## Getting TensorFlow Dataset

In [ ]:
# Getting TensorFlow train dataset

BATCH_SIZE = 4                            # (opt) Tested for our dataset to be the most suitable
data_size = train_data.shape[0]           # Number of data_points
train_dataset = tf.data.Dataset.from_tensor_slices(train_data).shuffle(data_size).batch(BATCH_SIZE)         # Shuffle and build the train dataset
print(train_dataset)

# Building and Training the GAN Model

## The Generator Model

In [ ]:
noise_dim = 100                   # Dimension of the input noise vector to the generator (opt)
feature_dim = train_data.shape[1]                      # Dimension of each feature (row)

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Input((noise_dim)))
      
    # Fully Connected Layers
    #(opt) (number of nodes can change and activation may be relu or leaky relu)
    
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))   
    
    model.add(layers.Dense(256, activation="leaky_relu"))    
    model.add(layers.Dense(feature_dim))
    model.compile()
    
    print(model.output_shape)
    assert model.output_shape == (None, feature_dim)               

    return model

In [ ]:
# Create an instance of the generator

generator = make_generator_model()
generator.summary()

In [ ]:
# Use the untrained system to create one sample output

noise = tf.random.normal([1, noise_dim])
generated_data = generator(noise, training=False)
generated_data_ = generated_data.numpy().reshape(-1).tolist()
plt.plot(generated_data_)

## The Discriminator Model

In [ ]:
def make_discriminator_model():
    
    # Implementing a ConvNet discriminator
    model = tf.keras.Sequential()
    
    model.add(layers.Input(shape= (feature_dim)))
    model.add(layers.Reshape([feature_dim, 1]))
    model.add(layers.Conv1D(kernel_size= 15, filters= 256, activation='leaky_relu'))  #(opt) (number of filters and kernel size)
    model.add(layers.MaxPool1D())
    model.add(layers.Dropout(0.2))                                                      #(opt) (dropout probability)
    
    model.add(layers.Conv1D(kernel_size= 15, filters= 128))    #(opt) (number of filters and kernel size)
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
        
    model.add(layers.MaxPool1D())
    model.add(layers.Dropout(0.2))                                                      #(opt) (dropout probability)

    model.add(layers.Flatten())
    model.add(layers.Dense(64))                                                         #(opt) (number of nodes in layer)
    model.add(layers.Dense(1))
    model.compile()
    
    return model

In [ ]:
# Create an instance of the discriminator model

discriminator = make_discriminator_model()          
decision = discriminator(generated_data)            # Get real or fake for the input we just got out of the generator
print (decision)                                
generator.summary()

## Defining Losses and Optimizers

In [ ]:
# Computation of cross entropy loss

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Definining the discriminator loss

def discriminator_loss(real_output, fake_output):
    return cross_entropy(tf.ones_like(real_output), real_output) + cross_entropy(tf.zeros_like(fake_output), fake_output)

# Defining the generator loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# Defining training optimizers

generator_optimizer = tf.keras.optimizers.Adam(1e-5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)

In [ ]:
# Model checkpoints saving

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Functions for Visualization and Data Generation

In [ ]:
seed = tf.random.normal([1, noise_dim])   # Fixed input noise distribution to monitor training effect on that distribution            

def draw_training_evolution(model, epoch, noise_input= seed):
  """
    Function that takes in the generator model, epoch number, and 
    does a prediction and plots the generated singal then saves it.
  """
  # `training` is set to False.
  # Thus, all layers run in inference mode (batchnorm).
  predictions = model(noise_input, training=False)
  
  for i in range(predictions.shape[0]):
    fig = plt.figure()
    plt.plot(predictions[i].numpy().tolist())
    plt.savefig('output/evolution/image_at_epoch_{:04d}_{:04d}.png'.format(epoch, i))
    plt.close()

def generate_data(model, num_synthetic_to_gen=1):
  """
    Function that takes in the generator model and 
    does a prediction and returns it as a numpy array.
  """
  noise_input = tf.random.normal([num_synthetic_to_gen, noise_dim])
  predictions = model(noise_input, training=False)
  predictions = predictions.numpy()
  return predictions

def calc_accuracy(prediction):
  """
    Function that takes in the some data judgements 
    from the discriminator and get the average of 
    judgements that indicate how the discriminator is fooled.
  """
  prediction_clipped = tf.clip_by_value(prediction, 0.0, 1.0, name=None)
  return tf.reduce_mean(prediction_clipped)

## Training the GAN Model

In [ ]:
# `tf.function` # This annotation causes the function to be "compiled".
@tf.function
def train_step(data):
    """
      Function for implementing one training step 
      of the GAN model
    """
    noise = tf.random.normal([BATCH_SIZE, noise_dim], seed=1)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_data = generator(noise, training=True)

      real_output = discriminator(data, training=True)
      fake_output = discriminator(generated_data, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
      acc = calc_accuracy(fake_output)
     
        

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss, acc

In [ ]:
numofEPOCHS = 10000 #(opt)

In [ ]:
def train(dataset, epochs):
  """
    Main GAN Training Function
  """
  epochs_gen_losses, epochs_disc_losses, epochs_accuracies = [], [], []

  for epoch in range(epochs):
    start = time.time()

    gen_losses, disc_losses, accuracies = [], [], []

    for data_batch in dataset:
      gen_loss, disc_loss, acc = train_step(data_batch)
      accuracies.append(acc)
      gen_losses.append(gen_loss)
      disc_losses.append(disc_loss)

    epoch_gen_loss  = np.average(gen_losses)
    epoch_disc_loss = np.average(disc_losses)
    epoch_accuracy = np.average(accuracies)
    epochs_gen_losses.append(epoch_gen_loss)
    epochs_disc_losses.append(epoch_disc_loss)
    epochs_accuracies.append(epoch_accuracy)
    print("Epoch: {}/{}".format(epoch+1, epochs))
    print("Generator Loss: {}, Discriminator Loss: {}".format(epoch_gen_loss, epoch_disc_loss))
    print("Accuracy: {}".format(epoch_accuracy))
      
    # Draw the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      draw_training_evolution(generator, epoch+1)
        
    # Save the model every 2 epochs for the last 2000 epochs
    if (epoch + 1) % 2 == 0 and epoch > (numofEPOCHS - 2000):
      checkpoint.save(file_prefix = checkpoint_prefix)   # Comment not to save model checkpoints while training
      

  return epochs_gen_losses, epochs_disc_losses, epochs_accuracies   

## Run

In [ ]:
EPOCHS = numofEPOCHS

epochs_gen_losses, epochs_disc_losses, epochs_accuracies = train(train_dataset, EPOCHS)


# Evaluating Model and Calculation of Performance Metrics

## Plotting Training Curves (Loss)

In [ ]:
plt.figure()
plt.rcParams.update({'font.size': 20}) # must be set on top
ax = pd.DataFrame(
    {
        'Generative Loss': epochs_gen_losses,
        'Discriminative Loss': epochs_disc_losses,
    }
).plot(title='Training loss', logy=True, figsize=(18,12))
ax.set_xlabel("Epochs", fontsize=18)
ax.set_ylabel("Loss", fontsize=18)

# Save figure using 600 dpi
plt.savefig("training.png", dpi=600)
plt.show()

## Model saving

In [ ]:
# Save the model
generator.save('models/generator_model23o.ckpt', overwrite=True)

## Calculation of Performance Metrics

### Root Mean Squared Error

In [ ]:
def get_rmse(synthetic_datapoint, original_datapoint):
    """
        Function that calculates the RMS between two datapoints
    """
    return np.sqrt(np.average((synthetic_datapoint - original_datapoint)**2))


def get_rmse_on_batch(synthetic_data, test_dataset):
    """
        Function that calculates the minimum RMS between 
        a batch of synthetic datapoints and a batch of test samples
    """
    
    rmse_all = []

    for gen in synthetic_data:
        rmse = np.inf
        for test_datapoint in test_dataset:
            current_rmse = get_rmse(gen, test_datapoint)
            if current_rmse < rmse:
                rmse = current_rmse
        rmse_all.append(rmse)

    return np.average(rmse_all) 

### Percent Root Mean Square Difference


In [ ]:
def get_prmsd(synthetic_datapoint, original_datapoint):
    """
        Function that calculates the percent root mean square 
        difference between two datapoints
    """
    return np.sqrt(100 * (np.sum((synthetic_datapoint - original_datapoint)**2)) / (np.sum(synthetic_datapoint**2)))

def get_prmsd_on_batch(synthetic_data, test_dataset):
    """
        Function that calculates the minimum percent root mean square 
        difference between a batch of synthetic
        datapoints and a batch of test samples
    """
    
    prmsd_all = []

    for gen in synthetic_data:
        prmsd = np.inf
        for test_datapoint in test_dataset:
            current_prmsd = get_prmsd(gen, test_datapoint)
            if current_prmsd < prmsd:
                prmsd = current_prmsd
        prmsd_all.append(prmsd)

    return np.average(prmsd_all) 

### Mean Absolute Error

In [ ]:
def get_mae(synthetic_datapoint, original_datapoint):
    """
        Function that calculates the mean absolute
        error between two datapoints
    """
    return np.average(np.abs(synthetic_datapoint - original_datapoint))

def get_mae_on_batch(synthetic_data, test_dataset):
    """
        Function that calculates the minimum mean absolute
        error between a batch of synthetic datapoints and a batch of test samples
    """
    
    mae_all = []

    for gen in synthetic_data:
        mae = np.inf
        for test_datapoint in test_dataset:
            current_mae = get_mae(gen, test_datapoint)
            if current_mae < mae:
                mae = current_mae
        mae_all.append(mae)

    return np.average(mae_all) 

### All Performance Metrics Calculation

In [ ]:
batch = generate_data(generator, num_synthetic_to_gen= 10)
rmse_ = get_rmse_on_batch(batch, test_data)
prmsd_ = get_prmsd_on_batch(batch, test_data)
mae_ = get_mae_on_batch(batch, test_data)
print("RMSe at Testing Dataset: {}".format(rmse_))
print("PRMSD at Testing Dataset: {}".format(prmsd_))
print("MAE at Testing Dataset: {}".format(mae_))

# Saving Synthesized Data as CSV

In [ ]:
def save_data_to_csv(data, filepath):
    """
        Function that takes in the data as numpy array,
        converts to pandas dataframe and then saves the .csv file.
    """
   # columns = ["Column{}".format(i) for i in range(data.shape[1])]
    
   
    df = pd.DataFrame(data, columns= column_labels())
    df.to_csv(filepath)

def draw_generated_figures(data, folderpath):
    """
        Function that takes in the generated batch of data
        and saves the corresponding signal outputs as figures
    """
    for i in range(data.shape[0]):
        fig = plt.figure()
        plt.plot(data[i].tolist(), 'r')
        plt.savefig(folderpath + '/generated_sample_{:04d}.png'.format(i))
        plt.close()
    

In [ ]:
# Generating samples
generated_batch = generate_data(generator, num_synthetic_to_gen=100)

# Undoing any normalization that happened 
# generated_batch = ((generated_batch * std_dev) + means).astype(np.int32)     # Converting to ints
save_data_to_csv(generated_batch, 'output/csv/samples100.csv')
draw_generated_figures(generated_batch, 'output/generated_samples')
print("Saved successfully!")

# Using a Pretrained Model

In [ ]:
loaded_generator = tf.keras.models.load_model('models/generator_model_A.ckpt')        # Load the model
loaded_generator.compile()                                                          # Compile the model
generated_data = generate_data(generator, num_synthetic_to_gen=1)
print(generated_data)

## Loading and exporting the last 1000 checkpoints (in 2000 epochs)

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

In [ ]:
checkpoint.restore(latest)

In [ ]:
generator(noise)

In [ ]:
# Save the checkpoints as spectra
for x in range(1000):
    checkpoint.restore('./training_checkpoints/ckpt-' + str(x+1))
    checkpoint.generator(noise)
    generated_batch = generate_data(generator, num_synthetic_to_gen=1)
    save_data_to_csv(generated_batch, 'output/cp/synth_o_raw_bz4_bn_GN00_model23_cp' + str(x+1) + '.csv')
print("Saved successfully!")

In [ ]:
cwd = os.getcwd()
print(cwd) # Sanity check: being in the correct directory

In [ ]:
# Concatenate the files

os.chdir('./output/cp')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
#export to csv

combined_csv.to_csv('synth_o_raw_bz4_bn_GN00_model23_samples1000_cp.csv', index=False, encoding='utf-8-sig')
print("Saved successfully!")

In [ ]:
os.chdir(cwd) # return to the starting directory and...

## End